In [19]:
import numpy as np
import math
import os
import pandas as pd
import scipy.stats as ss
from datetime import datetime
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression

In [20]:
def locate_index(value,bin_min,step_size,precision):
  loc0 = (value-bin_min)/step_size
  if loc0 < 0:
    loc = 0
  elif loc0 >= precision:
    loc = precision-1
  else:
    loc = int(loc0)
  return loc

def shift_hist(old_hist,shift):
  if shift<0:
    left_end = sum(old_hist[:(abs(shift)+1)])
    rest = old_hist[(abs(shift)+1):]
    fill = np.zeros(len(old_hist)-1-len(rest))
    new_hist = np.concatenate(([left_end],rest,fill))
  elif shift>0:
    right_end = sum(old_hist[-(abs(shift)+1):])
    rest = old_hist[:-(abs(shift)+1)]
    fill = np.zeros(len(old_hist)-1-len(rest))
    new_hist = np.concatenate((fill,rest,[right_end]))
  else:
    new_hist = old_hist
  return new_hist

def hist2tilde(hist_j,loc):
  cumsum = np.cumsum(np.sum(hist_j,axis=0))
  n = cumsum[-1]
  percentile = (cumsum[loc]+1)/(n+2)
  eps_tilde_j = ss.norm.ppf(percentile)
  return eps_tilde_j

def eps2tilde(eps):
  temp = eps.argsort()
  ranks = np.empty_like(temp)
  ranks[temp] = np.arange(len(eps))
  eps_tilde = ss.norm.ppf((ranks+1)/(len(eps)+1))
  return eps_tilde

In [476]:
def initialize_hist(data,period,r,precision):
  hist_dict = {}
  (nrow,ncol) = data.shape
  test_stats = np.zeros(nrow)
  eps_tilde_mat = np.zeros((nrow,ncol))
  bin_min_vec = np.zeros(ncol)
  step_size_vec = np.zeros(ncol)
  day_sum_mat = np.zeros((period,ncol))
  temporal_index = np.arange(nrow)
  periodic_index = temporal_index%period
  count = np.array([sum(periodic_index==k) for k in range(period)])
  for j in range(ncol):
    feature = data[:,j]
    y_minus_mu = np.zeros(nrow) 
    eps_hist = np.zeros((period,precision))
    for i in range(nrow):
      if i<2*period:
        mu = np.mean(feature)
      else:
        w = ss.t.pdf(np.arange(i-1,0,step=-1),2,0,nrow/10)
        mu = sum(feature[np.arange(i-1)]*w)/sum(w)
      y_minus_mu[i] = feature[i]-mu
    day_sum = np.array([sum(y_minus_mu[periodic_index==k]) for k in range(period)])
    s = day_sum/count
    eps = np.array([y_minus_mu[k]-s[k%period] for k in range(nrow)])
    eps_tilde_mat[:,j] = eps2tilde(eps)
    diff = max(eps)-min(eps)
    bin_min = min(eps)-r*diff
    bin_min_vec[j] = bin_min
    bin_max = max(eps)+r*diff
    step_size = (bin_max-bin_min)/precision
    step_size_vec[j] = step_size
    for i in range(nrow):
      row_loc = i%period
      col_loc = locate_index(eps[i],bin_min,step_size,precision)
      eps_hist[row_loc, col_loc]=eps_hist[row_loc, col_loc]+1
    hist_dict[str(j)] = eps_hist
    day_sum_mat[:,j] = day_sum
  var = np.cov(eps_tilde_mat,rowvar=False)
  for i in range(nrow):
    test_stats[i]=eps_tilde_mat[i,:].dot(np.linalg.pinv(var)).dot(eps_tilde_mat[i,:])
  memory = data
  N = data.shape[0]
  return hist_dict,day_sum_mat,count,bin_min_vec,step_size_vec,var,memory,test_stats,N

In [477]:
def online_update(new_input,hist_dict,day_sum_mat,count,var,memory,max_storage,N):
  (period, ncol) = day_sum_mat.shape
  n = sum(count)
  eps_vec = np.zeros(ncol)
  count[n%period] = count[n%period] + 1
  for j in range(ncol):
    w = ss.t.pdf(np.arange(memory.shape[0],0,step=-1),2,0,memory.shape[0]/10)
    mu = sum(memory[:,j]*w)/sum(w)
    y_minus_mu = new_input[j]-mu
    old_avg = day_sum_mat[n%period,j]/(count[n%period]-1)
    day_sum_mat[n%period,j] = day_sum_mat[n%period,j] +  y_minus_mu
    new_avg = day_sum_mat[n%period,j]/count[n%period] 
    new_eps = y_minus_mu - new_avg
    loc = locate_index(new_eps,bin_min_vec[j],step_size_vec[j],precision)
    eps_vec[j] = hist2tilde(hist_dict[str(j)],loc)
    shift = int(np.round((old_avg-new_avg)/step_size_vec[j],0))
    old_hist = hist_dict[str(j)][n%period,:]
    new_hist = shift_hist(old_hist,shift)
    new_hist[loc] = new_hist[loc] + 1
    hist_dict[str(j)][n%period,:] = new_hist
  cov = n/(n+1)*var+1/(n+1)*np.outer(eps_vec,eps_vec)
  std = np.sqrt(np.diag(cov))
  var = cov/np.outer(std,std)
  var[cov==0] = 0
  stat = eps_vec.dot(np.linalg.pinv(var)).dot(eps_vec)
  if n < max_storage:
    memory = np.vstack((memory,np.reshape(new_input,(1,ncol))))
  else:
    memory = memory[1:,:]
    memory =  np.vstack((memory,np.reshape(new_input,(1,ncol))))
  N = N+1
  return stat,hist_dict,day_sum_mat,count,var,memory,N

In [139]:
def eval_perf(pred, truth):
  prec = sum((pred==1)*(truth==1))/sum(pred==1)
  recall = sum((pred==1)*(truth==1))/sum(truth==1)
  F1 = 2*prec*recall/(prec+recall)
  return [prec,recall,F1]

In [367]:
def mag_var(var):
  abs_sum = 0 
  raw_sum = 0
  count = var.shape[0]*(var.shape[0]-1)/2
  for i in np.arange(0,var.shape[0]-1):
    for j in np.arange(i+1,var.shape[0]):
      abs_sum = abs_sum + abs(var[i,j])
      raw_sum = raw_sum + var[i,j]
  return abs_sum/count, raw_sum/count

In [368]:
## find relationship, independent X
sim_tab = []
for D in [2,5,10,20,30,40,50,60,70]:
  for rep in range(5):
    K = 1000
    data = np.zeros((K,D))
    for i in range(D):
      scale = np.random.uniform(1e-6,3,size=1)[0]
      loc = np.random.uniform(0,7,size=1)[0]
      data[:,i] = np.array([scale*np.sin(2*math.pi/7*(x+loc)) for x in range(K)]) + np.random.normal(0,scale,K)
    initial_data = data[:100,:]
    train_data = data[100:,]
    stat_vec = []
    hist_dict,day_sum_mat,count,bin_min_vec,step_size_vec,var,memory = initialize_hist(initial_data,period,r,precision)
    for i in range(train_data.shape[0]):
      new_input = train_data[i,:]
      stat,hist_dict,day_sum_mat,count,var,memory = online_update(new_input,hist_dict,day_sum_mat,count,var,memory,max_storage)
      stat_vec.append(stat)
    p95 = np.percentile(np.array(stat_vec),95)
    abs_var, raw_var = mag_var(var)
    sim_tab.append([D,rep,p95,ss.chi2.ppf(0.95,D),abs_var,raw_var])
sim_tab = np.array(sim_tab)

In [382]:
## correlated
sim_tab = []
for D in [2,5,10,20,30,40,50,60,70]:
  for rep in range(20):
    K = 1000
    data = np.zeros((K,D))
    for i in range(D):
      scale = np.random.uniform(1e-6,3,size=1)[0]
      loc = np.random.uniform(0,7,size=1)[0]
      if i>0:
        w = np.random.uniform(-0.7,0.7,size=1)[0]
        data[:,i] = (1-w)*(np.array([scale*np.sin(2*math.pi/7*(x+loc)) for x in range(K)]) + np.random.normal(0,scale,K))+w*data[:,i-1]
      else:
        data[:,i] = np.array([scale*np.sin(2*math.pi/7*(x+loc)) for x in range(K)]) + np.random.normal(0,scale,K)
    initial_data = data[:100,:]
    train_data = data[100:,]
    stat_vec = []
    hist_dict,day_sum_mat,count,bin_min_vec,step_size_vec,var,memory = initialize_hist(initial_data,period,r,precision)
    for i in range(train_data.shape[0]):
      new_input = train_data[i,:]
      stat,hist_dict,day_sum_mat,count,var,memory = online_update(new_input,hist_dict,day_sum_mat,count,var,memory,max_storage)
      stat_vec.append(stat)
    p95 = np.percentile(np.array(stat_vec),95)
    abs_var, raw_var = mag_var(var)
    sim_tab.append([D,rep,p95,ss.chi2.ppf(0.95,D),abs_var,raw_var])
sim_tab = np.array(sim_tab)
pd.DataFrame(sim_tab).to_csv("C:/Users/glius/Downloads/sim_cor.csv")

In [389]:
## three predictors abs_var, D and ss.chi2.ppf(0.95,D)
def find_threshold(var,D):
  abs_sum = 0 
  count = D*(D-1)/2
  for i in np.arange(0,D-1):
    for j in np.arange(i+1,D):
      abs_sum = abs_sum + abs(var[i,j])
  mean_cor = abs_sum/count
  chi2 = ss.chi2.ppf(0.95,D)
  threshold = 0.1877 + 0.0334*D + 0.9231*chi2 + 0.6314*mean_cor - 0.1763*chi2*mean_cor 
  return threshold

In [449]:
K = 6000
D = 20
p = 0.03
data = np.zeros((K,D))
for i in range(D):
  scale = np.random.uniform(1e-6,3,size=1)[0]
  loc = np.random.uniform(0,7,size=1)[0]
  if i>0:
    w = np.random.uniform(0.2,0.8,size=1)[0]
    data[:,i] = w*(np.array([scale*np.sin(2*math.pi/7*(x+loc)) for x in range(K)]) + np.random.normal(0,scale,K))+(1-w)*data[:,i-1]
  else:
    data[:,i] = np.array([scale*np.sin(2*math.pi/7*(x+loc)) for x in range(K)]) + np.random.normal(0,scale,K)
anomaly_index = np.sort(np.random.choice(np.arange(K), size=int(K*p), replace= False))
for i in anomaly_index:
  aberrant_num = np.random.choice(np.arange(int(D*0.3),int(D*0.7)))
  aberrant_col = np.random.choice(np.arange(D),aberrant_num,replace = False)
  data[i,aberrant_col] = data[i,aberrant_col]*np.random.uniform(0.33,3,size=aberrant_num)
initial_data = data[:100,:]
train_data = data[np.arange(100,int(K/2)),:]
test_data = data[int(K/2):,:]
labels = np.zeros(K)
labels[anomaly_index[anomaly_index>=int(K/2)]] = 1
train_labels = labels[:int(K/2)]
test_labels = labels[int(K/2):]
mad_gan_train = np.hstack((np.vstack((initial_data,train_data)),train_labels.reshape(-1,1)))
mad_gan_test = np.hstack((test_data,test_labels.reshape(-1,1)))

In [450]:
np.save('C:/Users/glius/Downloads/MAD-GANs-master/data/sim_train.npy',mad_gan_train)
np.save('C:/Users/glius/Downloads/MAD-GANs-master/data/sim_test.npy',mad_gan_test)
mad_gan_train.shape

(3000, 21)

In [478]:
## simulation
period = 7
r = 0.15
precision = 1000
max_storage = 1000
g = 100
h = 4

results = []
for K in [2000,4000,8000]:
  for D in [20, 40, 80]:
    for p in [0.005, 0.015, 0.045]:
      data = np.zeros((K,D))
      for i in range(D):
        scale = np.random.uniform(1e-6,3,size=1)[0]
        loc = np.random.uniform(0,7,size=1)[0]
        if i>0:
          w = np.random.uniform(0.2,0.8,size=1)[0]
          data[:,i] = w*(np.array([scale*np.sin(2*math.pi/7*(x+loc)) for x in range(K)]) + np.random.normal(0,scale,K))+(1-w)*data[:,i-1]
        else:
          data[:,i] = np.array([scale*np.sin(2*math.pi/7*(x+loc)) for x in range(K)]) + np.random.normal(0,scale,K)
      anomaly_index = np.sort(np.random.choice(np.arange(K), size=int(K*p), replace= False))
      for i in anomaly_index:
        aberrant_num = np.random.choice(np.arange(int(D*0.3),int(D*0.7)))
        aberrant_col = np.random.choice(np.arange(D),aberrant_num,replace = False)
        data[i,aberrant_col] = data[i,aberrant_col]*np.random.uniform(0.33,3,size=aberrant_num)
      initial_data = data[:100,:]
      train_data = data[np.arange(100,int(K/2)),:]
      test_data = data[int(K/2):,:]
      labels = np.zeros(K)
      labels[anomaly_index[anomaly_index>=int(K/2)]] = 1
      train_labels = labels[:int(K/2)]
      test_labels = labels[int(K/2):]
      
      anomaly_hist = np.zeros(g)
      t0 = datetime.now()
      hist_dict,day_sum_mat,count,bin_min_vec,step_size_vec,var,memory,test_stats,N = initialize_hist(initial_data,period,r,precision)
      threshold95 = find_threshold(var,D)
      step = threshold95/(D*2)
      found = 0
      for i in range(N):
        if test_stats[i]>threshold95:
          index = min(int((test_stats[i]-threshold95)/step),g-1)
          anomaly_hist[index] = anomaly_hist[index] + 1
      
      reverse_cumsum =  np.cumsum(np.flip(anomaly_hist,axis=0))
      if sum(anomaly_hist)>N*0.05:
        found = 1
        reverse_count = int(np.ceil(sum(anomaly_hist)-N*0.05))
        if reverse_cumsum[0]>reverse_count+1:
          index = 0
        else:
          index = np.where(reverse_cumsum>=reverse_count+1)[0][0]
        threshold_anomaly = threshold95 + (g-index)*step
      else:
        observed = 1
        for i in np.arange(1,g):
          if reverse_cumsum[i]!=reverse_cumsum[i-1]:
            observed = observed + 1
            if observed == h or i==g-1:
              threshold_anomaly = threshold95 + (g-i)*step
              break
 

      for i in range(train_data.shape[0]):
        new_input = train_data[i,:]
        stat,hist_dict,day_sum_mat,count,var,memory,N = online_update(new_input,hist_dict,day_sum_mat,count,var,memory,max_storage,N)
        if stat>threshold95:
          index = min(int((stat-threshold95)/step),g-1)
          anomaly_hist[index] = anomaly_hist[index] + 1
        
        reverse_cumsum =  np.cumsum(np.flip(anomaly_hist,axis=0))
        if sum(anomaly_hist)>N*0.05:
          found = 1
          reverse_count = int(np.ceil(sum(anomaly_hist)-N*0.05))
          if reverse_cumsum[0]>reverse_count+1:
            index = 0
          else:
            index = np.where(reverse_cumsum>=reverse_count+1)[0][0]
          threshold_anomaly = threshold95 + (g-index)*step
        elif found==0:
          observed = 1
          for i in np.arange(1,g):
            if reverse_cumsum[i]!=reverse_cumsum[i-1]:
              observed = observed + 1
              if observed == h or i == g-1:
                threshold_anomaly = threshold95 + (g-i)*step
                break
        else:
          threshold_anomaly = threshold_anomaly
          
      
      pred_fly = []
      stat_retro = []
      for i in range(test_data.shape[0]):
        new_input = test_data[i,:]
        stat,hist_dict,day_sum_mat,count,var,memory,N = online_update(new_input,hist_dict,day_sum_mat,count,var,memory,max_storage,N)
        if stat>threshold95:
          index = min(int((stat-threshold95)/step),g-1)
          anomaly_hist[index] = anomaly_hist[index] + 1
       
        reverse_cumsum =  np.cumsum(np.flip(anomaly_hist,axis=0))
        if sum(anomaly_hist)>N*0.05:
          found = 1
          reverse_count = int(np.ceil(sum(anomaly_hist)-N*0.05))
          if reverse_cumsum[0]>reverse_count+1:
            index = 0
          else:
            index = np.where(reverse_cumsum>=reverse_count+1)[0][0]
          threshold_anomaly = threshold95 + (g-index)*step
        elif found==0:
          observed = 1
          for i in np.arange(1,g):
            if reverse_cumsum[i]!=reverse_cumsum[i-1]:
              observed = observed + 1
              if observed == h or i==g-1:
                threshold_anomaly = threshold95 + (g-i)*step
                break
        else:
          threshold_anomaly = threshold_anomaly
        pred_fly.append(stat>threshold_anomaly)
        stat_retro.append(stat)
        
      t1 = datetime.now()
      t_need = (t1-t0).total_seconds()
      pred_fly = np.array(pred_fly)*1
      pred_retro = (np.array(stat_retro)>threshold_anomaly)*1
      eval_fly = eval_perf(pred_fly, test_labels)
      eval_retro = eval_perf(pred_retro, test_labels)
      print([K,D,p,eval_fly[0],eval_fly[1],eval_fly[2],eval_retro[0],eval_retro[1],eval_retro[2],t_need])
      results.append([K,D,p,eval_fly[0],eval_fly[1],eval_fly[2],eval_retro[0],eval_retro[1],eval_retro[2],t_need])
      temp = np.array(results)
      np.save("C:/Users/glius/Downloads/sim_result.npy",temp)

[2000, 20, 0.005, 0.15384615384615385, 0.5, 0.23529411764705882, 0.15384615384615385, 0.5, 0.23529411764705882, 20.569292]
[2000, 20, 0.015, 1.0, 0.5833333333333334, 0.7368421052631579, 1.0, 0.5833333333333334, 0.7368421052631579, 20.239179]
[2000, 20, 0.045, 1.0, 0.2631578947368421, 0.4166666666666667, 1.0, 0.2631578947368421, 0.4166666666666667, 20.494994]
[2000, 40, 0.005, 0.3684210526315789, 1.0, 0.5384615384615384, 0.3684210526315789, 1.0, 0.5384615384615384, 40.541098]
[2000, 40, 0.015, 0.8181818181818182, 0.5625, 0.6666666666666666, 0.8571428571428571, 0.75, 0.7999999999999999, 41.579796]
[2000, 40, 0.045, 1.0, 0.9512195121951219, 0.975, 1.0, 0.9512195121951219, 0.975, 40.806535]
[2000, 80, 0.005, 0.8571428571428571, 1.0, 0.923076923076923, 0.6, 1.0, 0.7499999999999999, 100.426506]
[2000, 80, 0.015, 1.0, 1.0, 1.0, 1.0, 1.0, 1.0, 99.19761]
[2000, 80, 0.045, 1.0, 0.9130434782608695, 0.9545454545454545, 1.0, 0.9130434782608695, 0.9545454545454545, 99.970841]
[4000, 20, 0.005, 0.562

KeyboardInterrupt: 